In [1]:
import numpy as np
import scipy
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Statevector, state_fidelity, Operator
from qiskit.synthesis import SuzukiTrotter
from qiskit_aer import AerSimulator
from qiskit_algorithms import TimeEvolutionProblem, TrotterQRTE
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.primitives import Estimator
from qiskit.circuit.library import PauliEvolutionGate
import matplotlib.pyplot as plt
from scipy.linalg import expm, cosm
import warnings
import sys
import functools
import make_circuit
import scipy as sc

In [10]:
initial_state = Statevector.from_label('00000')
step = 10
time = 1
time_step = 20
nqubits = 4
periodic = True
J = 1/np.sqrt(2)
def get_hamiltonian_y(nqubits, J, periodic):
    nqubits = nqubits - 1
    if periodic==False:
        ZZ_tuples = [('ZZY', [i, i+1, 0], J) for i in range(1, nqubits)]
        X_tuples = [("XY", [i, 0], J) for i in range(1, nqubits+1)]
        hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits+1)
    else:
        ZZ_tuples = [('ZZY', [i, i+1, 0], J) for i in range(1, nqubits)]
        ZZ_tuples += [('ZZY', [nqubits, 1, 0], J)]
        X_tuples = [("XY", [i, 0], J) for i in range(1, nqubits+1)]
        hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits+1)
    return hamiltonian.simplify()


def get_hamiltonian_i(nqubits, J, periodic):
    nqubits = nqubits - 1
    if periodic==False:
        ZZ_tuples = [('ZZI', [i, i+1, 0], J) for i in range(1, nqubits)]
        X_tuples = [("XI", [i, 0], J) for i in range(1, nqubits+1)]
        hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits+1)
    else:
        ZZ_tuples = [('ZZI', [i, i+1, 0], J) for i in range(1, nqubits)]
        ZZ_tuples += [('ZZI', [nqubits, 1, 0], J)]
        X_tuples = [("XI", [i, 0], J) for i in range(1, nqubits+1)]
        hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits+1)
    return hamiltonian.simplify()

def get_hamiltonian(nqubits, J):
    ZZ_tuples = [('ZZ', [i, i+1], J) for i in range(nqubits-1)]
    ZZ_tuples += [('ZZ', [nqubits-1, 0], J)]
    X_tuples = [("X", [i], J) for i in range(nqubits)]
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits)
    return hamiltonian.simplify()

In [ ]:
def unitary_trotter(nqubits, step, time, time_step, order):
    expectation_list = []
    J = 1/np.sqrt(2)
    #hamiltonian1 = make_heis(nqubits, periodic=True)
    hamiltonian1 = get_hamiltonian_y(nqubits+1, J, periodic=True)
    hamiltonian = get_hamiltonian_i(nqubits+1, J, periodic=True)
    hamiltonian2 = get_hamiltonian(nqubits, J) ## hamiltonian to calculate expectation 
    if order == 1:
        trotter= TrotterQRTE(num_timesteps=time_step, estimator=Estimator())
    else:
        trotter = TrotterQRTE(SuzukiTrotter(order=order), num_timesteps=time_step, estimator=Estimator())
    initial_state = Statevector.from_label('00000')
    #qc.initialize(initial_state, range(5))
    for i in range(step):
        problem = TimeEvolutionProblem(
            hamiltonian1,
            initial_state=initial_state,
            time=time,)
            #aux_operators=[hamiltonian],)
        result = trotter.evolve(problem)
        state = Statevector(result.evolved_state).data[:2**nqubits]
        state = state/np.linalg.norm(state)
        expectation = (state.conj().dot(hamiltonian2.to_matrix())).dot(state)
        expectation_list.append(expectation)
        #temp = np.kron(np.array([[1, 0]]), state/np.linalg.norm(state))
        temp = np.kron(state/np.linalg.norm(state), np.array([[1, 0]]))
        initial_state = Statevector(temp.reshape((temp.size,)))
    return expectation_list


def unitary_exact(nqubits, ):
    initial_state = Statevector.from_label('00000')
    state = initial_state.evolve(sc.linalg.expm(-1j * time * H_array))
    

In [11]:
get_hamiltonian(4, 1/np.sqrt(2))

SparsePauliOp(['IIZZ', 'IZZI', 'ZZII', 'ZIIZ', 'IIIX', 'IIXI', 'IXII', 'XIII'],
              coeffs=[0.70710678+0.j, 0.70710678+0.j, 0.70710678+0.j, 0.70710678+0.j, 0.70710678+0.j, 0.70710678+0.j, 0.70710678+0.j, 0.70710678+0.j])